In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
PREPARED_DATASET_PATH = 'joined_dataset_pollution_2.csv'
PREPARED_DATASET_PATH_2 = 'joined_dataset_COVID_2.csv'

Wall time: 0 ns


In [3]:
%%time
df_2 = pd.read_csv(PREPARED_DATASET_PATH_2)
df_2

Wall time: 2.89 s


,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800
1,37157.0,Rockingham,North Carolina,US,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800
2,37159.0,Rowan,North Carolina,US,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800
3,37161.0,Rutherford,North Carolina,US,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800
4,37163.0,Sampson,North Carolina,US,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,US,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000
1454382,17125.0,Mason,Illinois,US,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000
1454383,17127.0,Massac,Illinois,US,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000
1454384,17095.0,Knox,Illinois,US,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000


Унифицируем названия стран Дуграммы (сокращенное название) стран в обоих датасетах, 
вводим в оба датасета 1) поля название страны и 2) Дуграмма:

In [4]:
#подгружаем датафрейм с Дуграмами и названиями стран
df_00 = pd.read_excel('countries_alpha.xlsx')
df_00

,Country,1_Full_country_name,2_Full_country_name,Capital
0,AB,Abkhazia,Abkhazia,NaN
1,AF,Afghanistan,Afghanistan,Kabul
2,AX,Åland Islands,Åland Islands,NaN
3,AL,Albania,Albania,Tirana
4,DZ,Algeria,Algeria,Algiers
...,...,...,...,...
251,CI,Ivory Coast,Cote d'Ivoire,Yamoussoukro
252,GB,United Kingdom,Great Britain,London
253,TL,Timor-Leste,East Timor,Dili
254,RU,Russia,Russian Federation,Moscow


In [5]:
df_00.loc[df_00['Country'] == 'GB']

,Country,1_Full_country_name,2_Full_country_name,Capital
252,GB,United Kingdom,Great Britain,London


In [6]:
# видим проблему с 'Namibia' - дуграмма NA вопринимается как пустое значение. Заменяем его на тестовове "NA"
# Надо будет мониторить - появится ли эта ошибка при последующем созранении фала в csv и потом его вторичной  подгрузке в датасет
df_00.loc[df_00['1_Full_country_name'] == 'Namibia']

,Country,1_Full_country_name,2_Full_country_name,Capital
155,NaN,Namibia,Namibia,Windhoek


In [7]:
 df_00.loc[(df_00['1_Full_country_name'] == 'Namibia' ) , 'Country'] =  'NA'    

In [8]:
df_00.loc[df_00['1_Full_country_name'] == 'Namibia']

,Country,1_Full_country_name,2_Full_country_name,Capital
155,NA,Namibia,Namibia,Windhoek


In [9]:
# смотрим в чем разница между 1_Full_country_name и 2_Full_country_name

df_00_differ = df_00.loc[df_00['1_Full_country_name'] != df_00['2_Full_country_name']]
df_00_differ

,Country,1_Full_country_name,2_Full_country_name,Capital
27,BO,Bolivia,"Bolivia, plurinational state of",Sucre
34,BN,Brunei,Brunei Darussalam,Bandar Seri Begawan
37,MM,Burma,Myanmar,Naypyidaw
52,CG,Republic of the Congo,Congo,Brazzaville
53,CD,Democratic Republic of Congo,"Congo, Democratic Republic of the",Kinshasa
56,CI,Cote d'Ivoire,Ivory Coast,Yamoussoukro
59,CW,Curacao,Curaçao,NaN
68,SV,El Salvador,Salvador,San Salvador
72,SZ,Eswatini,Swaziland,Mbabane
76,FJ,Fiji,Fiji Islands,Suva


In [10]:
# !!! Тут иногда виснет Jupiter - надо перестартовать ядро 
# делаем словарь для замены через фукнцию multiple_replace
df_00_differ_dict = dict(zip(df_00_differ['1_Full_country_name'], df_00_differ['2_Full_country_name']))
df_00_differ_dict

{'Bolivia': 'Bolivia, plurinational state of',
 'Brunei': 'Brunei Darussalam',
 'Burma': 'Myanmar',
 'Republic of the Congo': 'Congo',
 'Democratic Republic of Congo': 'Congo, Democratic Republic of\xa0the',
 "Cote d'Ivoire": 'Ivory Coast',
 'Curacao': 'Curaçao',
 'El Salvador': 'Salvador',
 'Eswatini': 'Swaziland',
 'Fiji': 'Fiji Islands',
 'Vatican': 'Holy See (Vatican City State)',
 'Iran': 'Iran, Islamic Republic of',
 'North Korea': "Korea, Democratic People's republic of",
 'South Korea': 'Korea, Republic of',
 'Laos': "Lao People's Democratic Republic",
 'Libya': 'Libyan Arab Jamahiriya',
 'Macedonia': 'Macedonia, The Former Yugoslav Republic Of',
 'Marshall Islands': 'the Marshall Islands',
 'Federated States of Micronesia': 'Micronesia, Federated States of',
 'Moldavia': 'Moldova',
 'Netherlands': 'The Netherlands',
 'Philippines': 'The Philippines',
 'Russian Federation': 'Russia',
 'Saint Barthelemy': 'Saint Barthélemy',
 'Saint Martin': 'Saint Martin (French Part)',
 'Saint

In [11]:
#Уникальные страны в датафрейм joined_dataset_COVID_2
cntrs_unique_COVID = df_2['Country_Region'].unique()
print('Уникальных стран в joined_dataset_COVID_2',len(cntrs_unique_COVID))
cntrs_unique_df_00_1 = df_00['1_Full_country_name'].unique()
print('Уникальных стран в countries_alpha первый столбец',len(cntrs_unique_df_00_1))
cntrs_unique_df_00_2 = df_00['2_Full_country_name'].unique()
print('Уникальных стран в countries_alpha второй столбец',len(cntrs_unique_df_00_2))

Уникальных стран в joined_dataset_COVID_2 196
Уникальных стран в countries_alpha первый столбец 256
Уникальных стран в countries_alpha второй столбец 256


In [12]:
# Какие страны из датасет joined_dataset_COVID_2 не попали в список из countries_alpha
cntrs_not_included_1 = list(set(cntrs_unique_COVID) - set(cntrs_unique_df_00_1))
print(cntrs_not_included_1, '\n--Этих стран из joined_dataset_COVID_2 нет в countries_alpha первый столбец\n')
cntrs_not_included_2 = list(set(cntrs_unique_COVID) - set(cntrs_unique_df_00_2))
print(cntrs_not_included_2, '\n--Этих стран из joined_dataset_COVID_2 нет в countries_alpha второй столбец\n')

['North Macedonia', 'Micronesia', 'Czechia', 'Taiwan*', 'Summer Olympics 2020', 'Congo (Brazzaville)', 'US', 'Cabo Verde', 'MS Zaandam', 'Diamond Princess', 'Korea, South', 'Holy See', 'West Bank and Gaza', 'Congo (Kinshasa)'] 
--Этих стран из joined_dataset_COVID_2 нет в countries_alpha первый столбец

['North Macedonia', 'Laos', 'Samoa', 'Iran', 'Czechia', 'Philippines', 'Cabo Verde', 'Brunei', 'Netherlands', 'Bolivia', 'Korea, South', 'Marshall Islands', 'West Bank and Gaza', 'El Salvador', 'Syria', 'Congo (Brazzaville)', 'US', 'Libya', 'MS Zaandam', 'Diamond Princess', 'South Africa', 'Congo (Kinshasa)', 'Micronesia', 'Summer Olympics 2020', 'United Kingdom', 'Saint Vincent and the Grenadines', 'Fiji', 'Burma', 'Taiwan*', 'Tanzania', 'Eswatini', 'Holy See'] 
--Этих стран из joined_dataset_COVID_2 нет в countries_alpha второй столбец



Работаем с уникальными значениями стран и сравниваем все ли есть в countries_alpha. Цель - по каждой строчке в датасете joined_dataset_COVID_2 получить Дуграмму

In [13]:
# комментарии:
# Summer Olympics 2020 - данные по аккредитованым на Олимппийские Игры 2020
# Holy See - Ватикан
# MS Zaandam - Круизное судно
# Diamond Princess- Круизное судно
# Данные по круизным судам и Summer Olympics 2020 -- удаяляем, так как по ним нет данных в датасетах по загрязнению воздуха

In [14]:
# делаем второй словарь для замены через фукнцию multiple_replace в Country_Region в датасете в joined_dataset_COVID_2
# смотрим какие значения из списка cntrs_not_included_1 предтавлены в неправлбной транкрипции и меняем ее на правильную 
# в словаре ниже:
replace_values = {'US': 'The United States of America', 'Cabo Verde': 'Cape Verde', 
                  'Congo Brazzaville,': 'Republic of the Congo', 'Congo (Brazzaville)': 'Republic of the Congo',
                  'Congo Kinshasa,': 'Democratic Republic of Congo', 'Congo (Kinshasa)': 'Democratic Republic of Congo',
                  'Czechia': 'Czech Republic', 'Korea: South': 'South Korea', 'Korea, South': 'South Korea',
                  'North Macedonia': 'Macedonia', 
                  'Taiwan*': 'Taiwan, Province of China', 'Micronesia': 'Federated States of Micronesia', 
                  'Mainland China': 'China', 'Macau': 'Macao', 'Taiwan': 'Taiwan, Province of China', 
                  'Taiwan': 'Taiwan, Province of China', 
                  'UK': 'Great Britain', ' Azerbaijan': 'Azerbaijan', 'Vatican City': 'Vatican', 
                  'Republic of Ireland': 'Ireland', 'St. Martin': 'Saint Martin', 'Iran Islamic Republic of,': 'Iran', 
                  'Republic of Korea': 'North Korea', 'Hong Kong SAR': 'Hong Kong', 'Viet Nam': 'Vietnam', 
                  'Macao SAR': 'Macao', 'Republic of Moldova': 'Moldavia', 'The Bahamas': 'Bahamas', 
                  'The Gambia': 'Gambia', 'Gambia: The': 'Gambia', 'Bahamas: The': 'Bahamas', 'Holy See':'Vatican', 
                  'West Bank and Gaza':'Palestinian Territory'
                  }


In [15]:
# df_2.loc[df_2['Country_Region'] == 'Holy See']

Теперь меняем в Country_Region в датасете в joined_dataset_COVID_2 названия на "правильные" - те, которые есть в countries_alpha первый столбец. После этого мы сможем получить Дуграмму только по первому столбцу

In [16]:
def multiple_replace(name, replace_values = replace_values):
    # получаем заменяемое: подставляемое из словаря в цикле
    for i, j in replace_values.items():
        # меняем все target_str на подставляемое
        name = name.replace(i, j)
    return name

In [17]:
df_2[['Country_Region', 'Combined_Key']] = df_2[['Country_Region', 'Combined_Key']].apply(multiple_replace)
# df_2['Combined_Key'] = df_2['Combined_Key'].apply(multiple_replace)
df_2

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000


In [18]:
# ['MS Zaandam', 'Cabo Verde', 'Czechia', 'US', 'Congo (Brazzaville)', 'Micronesia', 'Diamond Princess', 'North Macedonia', 'West Bank and Gaza', 'Congo (Kinshasa)', 'Summer Olympics 2020', 'Holy See', 'Taiwan*', 'Korea, South'] 
# --Этих стран из joined_dataset_COVID_2 нет в countries_alpha первый столбец
df_2.loc[df_2['Country_Region'] == 'Czech Republic']

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time
2520,NaN,NaN,NaN,Czech Republic,2021-01-02,49.8175,15.473,732022,11711,600073.0,120238.0,Czech Republic,6835.589041,1.599815,1609534800
6538,NaN,NaN,NaN,Czech Republic,2021-01-03,49.8175,15.473,735469,11827,608128.0,115514.0,Czech Republic,6867.776975,1.608090,1609621200
10524,NaN,NaN,NaN,Czech Republic,2021-01-04,49.8175,15.473,740481,11960,612214.0,116307.0,Czech Republic,6914.578809,1.615166,1609707600
14511,NaN,NaN,NaN,Czech Republic,2021-01-05,49.8175,15.473,746714,12070,615083.0,119561.0,Czech Republic,6972.782287,1.616415,1609794000
18528,NaN,NaN,NaN,Czech Republic,2021-01-06,49.8175,15.473,759635,12257,625890.0,121488.0,Czech Republic,7093.438013,1.613538,1609880400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436958,NaN,NaN,NaN,Czech Republic,2021-12-27,49.8175,15.473,2445741,35749,NaN,NaN,Czech Republic,22838.221224,1.461684,1640552400
1440935,NaN,NaN,NaN,Czech Republic,2021-12-28,49.8175,15.473,2447758,35805,NaN,NaN,Czech Republic,22857.055881,1.462767,1640638800
1444959,NaN,NaN,NaN,Czech Republic,2021-12-29,49.8175,15.473,2454645,35911,NaN,NaN,Czech Republic,22921.366382,1.462981,1640725200
1448952,NaN,NaN,NaN,Czech Republic,2021-12-30,49.8175,15.473,2463780,35975,NaN,NaN,Czech Republic,23006.668608,1.460155,1640811600


Теперь вставляем Дуграмму в датасет joined_dataset_COVID_2

In [19]:
#Из датасета df_00 (countries_alpha) делаем два столбца - Дуграмма и  столбец 1_Full_country_name, 
# название 1_Full_country_name меняем на Country_Region
df_00_for_merge = df_00[['Country', '1_Full_country_name']]
df_00_for_merge.rename(columns={'1_Full_country_name': 'Country_Region'}, inplace=True)

df_00_for_merge

C:\Users\aanal\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Country,Country_Region
0,AB,Abkhazia
1,AF,Afghanistan
2,AX,Åland Islands
3,AL,Albania
4,DZ,Algeria
...,...,...
251,CI,Ivory Coast
252,GB,United Kingdom
253,TL,Timor-Leste
254,RU,Russia


In [20]:
df_00_for_merge.loc[df_00_for_merge['Country'] == 'GB']

,Country,Country_Region
252,GB,United Kingdom


In [21]:
df_2 = pd.merge(df_2,df_00_for_merge,on=['Country_Region'], how='left')
df_2

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800,AF
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800,US
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800,US
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800,US
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000,US
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000,US
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000,US
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000,US


In [22]:
df_2.loc[df_2['Country_Region'] == 'MS Zaandam']

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country
2847,NaN,NaN,NaN,MS Zaandam,2021-01-02,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609534800,NaN
6758,NaN,NaN,NaN,MS Zaandam,2021-01-03,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609621200,NaN
10745,NaN,NaN,NaN,MS Zaandam,2021-01-04,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609707600,NaN
14732,NaN,NaN,NaN,MS Zaandam,2021-01-05,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609794000,NaN
18720,NaN,NaN,NaN,MS Zaandam,2021-01-06,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609880400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437352,NaN,NaN,NaN,MS Zaandam,2021-12-27,NaN,NaN,9,2,NaN,NaN,MS Zaandam,NaN,22.222222,1640552400,NaN
1441344,NaN,NaN,NaN,MS Zaandam,2021-12-28,NaN,NaN,9,2,NaN,NaN,MS Zaandam,NaN,22.222222,1640638800,NaN
1445210,NaN,NaN,NaN,MS Zaandam,2021-12-29,NaN,NaN,9,2,NaN,NaN,MS Zaandam,NaN,22.222222,1640725200,NaN
1449249,NaN,NaN,NaN,MS Zaandam,2021-12-30,NaN,NaN,9,2,NaN,NaN,MS Zaandam,NaN,22.222222,1640811600,NaN


In [23]:
pd.isnull(df_2['Country']).sum()


1092

In [24]:
df_2['Country'].isnull().values.any()

True

In [25]:
df_90 = df_2[df_2['Country'].isnull()]
df_90

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country
1316,NaN,NaN,NaN,Summer Olympics 2020,2021-01-02,35.6491,139.7737,0,0,0.0,0.0,Summer Olympics 2020,NaN,NaN,1609534800,NaN
2585,NaN,NaN,NaN,Diamond Princess,2021-01-02,NaN,NaN,712,13,699.0,0.0,Diamond Princess,NaN,1.825843,1609534800,NaN
2847,NaN,NaN,NaN,MS Zaandam,2021-01-02,NaN,NaN,9,2,7.0,0.0,MS Zaandam,NaN,22.222222,1609534800,NaN
5302,NaN,NaN,NaN,Summer Olympics 2020,2021-01-03,35.6491,139.7737,0,0,0.0,0.0,Summer Olympics 2020,NaN,NaN,1609621200,NaN
6572,NaN,NaN,NaN,Diamond Princess,2021-01-03,NaN,NaN,712,13,699.0,0.0,Diamond Princess,NaN,1.825843,1609621200,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448987,NaN,NaN,NaN,Diamond Princess,2021-12-30,NaN,NaN,712,13,NaN,NaN,Diamond Princess,NaN,1.825843,1640811600,NaN
1449249,NaN,NaN,NaN,MS Zaandam,2021-12-30,NaN,NaN,9,2,NaN,NaN,MS Zaandam,NaN,22.222222,1640811600,NaN
1452426,NaN,NaN,NaN,Summer Olympics 2020,2021-12-31,35.6491,139.7737,865,0,NaN,NaN,Summer Olympics 2020,NaN,0.000000,1640898000,NaN
1452982,NaN,NaN,NaN,Diamond Princess,2021-12-31,NaN,NaN,712,13,NaN,NaN,Diamond Princess,NaN,1.825843,1640898000,NaN


In [26]:
df_90['Country_Region'].unique()

array(['Summer Olympics 2020', 'Diamond Princess', 'MS Zaandam'],
      dtype=object)

Убираем строчки ['Summer Olympics 2020', 'Diamond Princess', 'MS Zaandam'], которые имеют нулевые значения в столбце дуграмм и отсутвют в датасете Pollution

In [27]:
df_2.dropna(subset=['Country'], inplace=True)
df_2

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800,AF
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800,US
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800,US
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800,US
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000,US
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000,US
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000,US
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000,US


In [28]:
#проверка нулевых результатов в столбце Дуграмм Country:
df_90 = df_2[df_2['Country'].isnull()]
df_90

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country


# Теперь работаем с датасетом df полученном из joined_dataset_pollution_2

In [29]:
%%time
df = pd.read_csv(PREPARED_DATASET_PATH)
df

Wall time: 2.67 s


,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time
0,2021-01-01,FR,Saint-Étienne,temperature,96,1.0,2.0,1.0,1.68,1609448400
1,2021-01-01,PL,Katowice,so2,94,3.1,17.6,8.8,106.62,1609448400
2,2021-01-01,RS,Novi Sad,so2,71,3.3,30.5,9.0,224.44,1609448400
3,2021-01-01,KR,Busan,humidity,637,39.0,68.0,51.0,588.90,1609448400
4,2021-01-01,DE,Freiburg,wind-speed,43,0.2,3.7,1.9,11.93,1609448400
...,...,...,...,...,...,...,...,...,...,...
2122873,2021-12-31,BA,Tuzla,o3,117,0.4,17.8,3.4,147.35,1640898000
2122874,2021-12-31,AT,Linz,dew,39,6.0,10.4,9.5,12.66,1640898000
2122875,2021-12-31,AU,Darwin,no2,69,0.3,3.6,1.1,11.98,1640898000
2122876,2021-12-31,ES,Oviedo,humidity,261,31.0,91.3,61.0,1623.29,1640898000


In [30]:
# проверяем все ли дуграммы есть в countries_alpha
cntrs_df = df['Country'].unique()
cntrs_unique_df_dugr = df_00['Country'].unique()

In [31]:
cntrs_not_included_1 = list(set(cntrs_df) - set(cntrs_unique_df_dugr))
print('Количество дуграм из датасета из joined_dataset_pollution_2, которых нет в countries_alpha: \n', len(cntrs_not_included_1) )

Количество дуграм из датасета из joined_dataset_pollution_2, которых нет в countries_alpha: 
 0


Теперь добавялем столбец Country_Region в датафрйем joined_dataset_pollution_2

In [32]:
df = pd.merge(df,df_00_for_merge,on=['Country'], how='left')
df

,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region
0,2021-01-01,FR,Saint-Étienne,temperature,96,1.0,2.0,1.0,1.68,1609448400,France
1,2021-01-01,PL,Katowice,so2,94,3.1,17.6,8.8,106.62,1609448400,Poland
2,2021-01-01,RS,Novi Sad,so2,71,3.3,30.5,9.0,224.44,1609448400,Serbia
3,2021-01-01,KR,Busan,humidity,637,39.0,68.0,51.0,588.90,1609448400,South Korea
4,2021-01-01,DE,Freiburg,wind-speed,43,0.2,3.7,1.9,11.93,1609448400,Germany
...,...,...,...,...,...,...,...,...,...,...,...
2136876,2021-12-31,BA,Tuzla,o3,117,0.4,17.8,3.4,147.35,1640898000,Bosnia and Herzegovina
2136877,2021-12-31,AT,Linz,dew,39,6.0,10.4,9.5,12.66,1640898000,Austria
2136878,2021-12-31,AU,Darwin,no2,69,0.3,3.6,1.1,11.98,1640898000,Australia
2136879,2021-12-31,ES,Oviedo,humidity,261,31.0,91.3,61.0,1623.29,1640898000,Spain


# Как результат действий выше, мы получаем в обоих датасетах joined_dataset_pollution_2 и joined_dataset_COVID_2 колонки с унифицированными значениями дуграмм(стран) в колонке  Country и названий стран в колонке  Country_Region

# Теперь унифицируем населенные пункты в двух датасетах

Проблема в том, что если в случае с датасетом joined_dataset_pollution_2 мы имеем только одну колонку  названием города 'City', то в датасете joined_dataset_COVID_2 мы имеем колонки 'Admin2',	'Province_State' в которых присутвуют населеные пункты

In [33]:
poll_uniq_towns = df['City'].unique()
print('Уникальных городов в joined_dataset_pollution_2: ',len(poll_uniq_towns))
covid_Admin2_uniq_towns = df_2['Admin2'].unique()
print('Уникальных городов в Admin2 в  joined_dataset_COVID_2: ',len(covid_Admin2_uniq_towns))
covid_PS_uniq_towns = df_2['Province_State'].unique()
print('Уникальных городов в Province_State в  joined_dataset_COVID_2: ',len(covid_PS_uniq_towns))

Уникальных городов в joined_dataset_pollution_2:  610
Уникальных городов в Admin2 в  joined_dataset_COVID_2:  1926
Уникальных городов в Province_State в  joined_dataset_COVID_2:  593


In [34]:
# проверяем сколько городов имеют пересечния :
towns_not_included_poll_admin2 = list(set(poll_uniq_towns) - set(covid_Admin2_uniq_towns))
print('Количество городов из датасета из pollution , \nкоторых нет в Admin2: ', len(towns_not_included_poll_admin2), ',   которые есть: ', (len(poll_uniq_towns)-len(towns_not_included_poll_admin2)),'\n')
towns_not_included_poll_PS = list(set(poll_uniq_towns) - set(covid_PS_uniq_towns))
print('Количество городов из датасета из pollution , \nкоторых нет в Province_State: ', len(towns_not_included_poll_PS), ',   которые есть: ', (len(poll_uniq_towns)-len(towns_not_included_poll_PS)),'\n')
# print('Количество городов из датасета из pollution , которых нет в Province_State: \n', len(towns_not_included_poll_PS))
towns_not_included_Admin2 = list(set(covid_Admin2_uniq_towns) - set(poll_uniq_towns))
print('Количество городов из датасета из Admin2 , \nкоторых нет в pollution:  ', len(towns_not_included_Admin2), ',   которые есть: ', (len(covid_Admin2_uniq_towns)-len(towns_not_included_Admin2)),'\n')
# print('Количество городов из датасета из Admin2 , которых нет в pollution: \n', len(towns_not_included_Admin2))
towns_not_included_PS = list(set(covid_PS_uniq_towns) - set(poll_uniq_towns))
print('Количество городов из датасета из Province_State , \nкоторых нет в pollution:  ', len(towns_not_included_PS), ',   которые есть: ', (len(covid_PS_uniq_towns)-len(towns_not_included_PS)),'\n')
# print('Количество городов из датасета из Province_State , которых нет в pollution: \n', len(towns_not_included_PS))

Количество городов из датасета из pollution , 
которых нет в Admin2:  568 ,   которые есть:  42 

Количество городов из датасета из pollution , 
которых нет в Province_State:  565 ,   которые есть:  45 

Количество городов из датасета из Admin2 , 
которых нет в pollution:   1884 ,   которые есть:  42 

Количество городов из датасета из Province_State , 
которых нет в pollution:   548 ,   которые есть:  45 



In [35]:
# Города которые естьи и в pollution и в Province_State
towns_same_poll_COVID = list(set(poll_uniq_towns) & set(covid_PS_uniq_towns))
print(towns_same_poll_COVID)
print(len(towns_same_poll_COVID))

['Aguascalientes', 'Brussels', 'Delhi', 'Saitama', 'Shanghai', 'Groningen', 'Murcia', 'Tokyo', 'Niigata', 'Miyazaki', 'Kagoshima', 'Namur', 'Stockholm', 'Berlin', 'Lima', 'Chiba', 'Macau', 'Hong Kong', 'Puebla', 'Moscow', 'Kyoto', 'Chandigarh', 'Kochi', 'Uppsala', 'Madrid', 'Akita', 'Wakayama', 'Beijing', 'Nagano', 'Okayama', 'Kumamoto', 'Nagasaki', 'Hamburg', 'Osaka', 'Santander', 'Islamabad', 'Chongqing', 'Tianjin', 'Toyama', 'Shizuoka', 'Victoria', 'Utrecht', 'Oaxaca', 'Fukuoka', 'Hiroshima']
45


In [36]:
#уникальные города для ВСЕГО сета joined_dataset_COVID_2
unique_set_for_COVID = pd.unique(df_2[['Admin2', 'Province_State']].values.ravel('K'))
print('Уникальных городов в Province_State в  joined_dataset_COVID_2: ',len(unique_set_for_COVID))
print('Количество городов, которые встречаются и колонке Admin2 и в Province_State',len(unique_set_for_COVID) - len(covid_PS_uniq_towns) - len(covid_Admin2_uniq_towns))

Уникальных городов в Province_State в  joined_dataset_COVID_2:  2493
Количество городов, которые встречаются и колонке Admin2 и в Province_State -26


In [37]:
def compare_towns (name, poll_uniq_towns = poll_uniq_towns):
    if name in poll_uniq_towns:
        return 1
    else:
        return 0

In [38]:
%%time
df_2['intersection_poll_Admin2'] = df_2['Admin2'].apply(compare_towns)
df_2['intersection_poll_PS'] = df_2['Province_State'].apply(compare_towns)
df_2['intersection_poll'] = np.maximum(df_2['intersection_poll_Admin2'], df_2['intersection_poll_PS'])
df_2 = df_2.drop(['intersection_poll_Admin2', 'intersection_poll_PS'], axis = 1)
df_2

Wall time: 1min 3s


,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800,AF,0
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800,US,0
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800,US,0
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800,US,0
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000,US,0
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000,US,0
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000,US,0
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000,US,0


In [39]:
df_2.loc[(df_2['intersection_poll'] == 1) ]

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
30,37153.0,Richmond,North Carolina,The United States of America,2021-01-02,35.004177,-79.744910,2630,47,0.0,2583.0,"Richmond, North Carolina, US",5866.738049,1.787072,1609534800,US,1
34,37099.0,Jackson,North Carolina,The United States of America,2021-01-02,35.288143,-83.140934,2259,18,0.0,2241.0,"Jackson, North Carolina, US",5141.335518,0.796813,1609534800,US,1
41,37115.0,Madison,North Carolina,The United States of America,2021-01-02,35.854027,-82.713464,797,19,0.0,778.0,"Madison, North Carolina, US",3663.525626,2.383940,1609534800,US,1
83,39009.0,Athens,Ohio,The United States of America,2021-01-02,39.334256,-82.042786,3101,25,0.0,3076.0,"Athens, Ohio, US",4746.888729,0.806192,1609534800,US,1
128,36041.0,Hamilton,New York,The United States of America,2021-01-02,43.661466,-74.497220,112,1,0.0,111.0,"Hamilton, New York, US",2536.231884,0.892857,1609534800,US,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454196,18103.0,Miami,Indiana,The United States of America,2021-12-31,40.769410,-86.045250,7188,123,NaN,NaN,"Miami, Indiana, US",20238.765627,1.711185,1640898000,US,1
1454310,16065.0,Madison,Idaho,The United States of America,2021-12-31,43.781410,-111.653847,9362,42,NaN,NaN,"Madison, Idaho, US",23459.543438,0.448622,1640898000,US,1
1454324,17065.0,Hamilton,Illinois,The United States of America,2021-12-31,38.081636,-88.538834,1540,25,NaN,NaN,"Hamilton, Illinois, US",18974.864465,1.623377,1640898000,US,1
1454361,17077.0,Jackson,Illinois,The United States of America,2021-12-31,37.785952,-89.382326,9483,101,NaN,NaN,"Jackson, Illinois, US",16710.132159,1.065064,1640898000,US,1


Выбираем страны присутующие в обоих датасетах joined_dataset_pollution_2 и joined_dataset_COVID_2 

In [40]:
# уникальные страны для joined_dataset_COVID_2
dugr_cntrs_unique_COVID = df_2['Country'].unique()
print(dugr_cntrs_unique_COVID)
print(len(dugr_cntrs_unique_COVID))

['AF' 'US' 'UG' 'UA' 'PS' 'YE' 'ZM' 'ZW' 'CN' 'KI' 'PW' 'NZ' 'MY' 'TO'
 'VN' 'VE' 'UZ' 'AE' 'GB' 'UY' 'VU' 'SE' 'CH' 'SY' 'TW' 'TJ' 'TZ' 'TH'
 'TL' 'TG' 'TT' 'TN' 'TR' 'RU' 'ES' 'LK' 'SD' 'SR' 'SS' 'RW' 'KN' 'LC'
 'VC' 'SM' 'ST' 'SA' 'SN' 'RS' 'SC' 'SL' 'SG' 'SK' 'SI' 'SB' 'SO' 'ZA'
 'WS' 'CO' 'KM' 'CG' 'CD' 'CR' 'CI' 'HR' 'CU' 'CY' 'CZ' 'DK' 'DE' 'GH'
 'GR' 'GD' 'GT' 'GN' 'GW' 'GY' 'HT' 'VA' 'HN' 'HU' 'IS' 'IN' 'GE' 'DJ'
 'DM' 'DO' 'EC' 'EG' 'SV' 'GQ' 'ER' 'EE' 'SZ' 'ET' 'FJ' 'FI' 'FR' 'GA'
 'GM' 'BE' 'BZ' 'BJ' 'BT' 'BO' 'BA' 'BW' 'BR' 'AL' 'AD' 'AO' 'AG' 'AR'
 'AM' 'AU' 'AT' 'AZ' 'BS' 'BH' 'BD' 'BB' 'BY' 'CL' 'TD' 'BN' 'BG' 'BF'
 'MM' 'BI' 'CV' 'KH' 'CM' 'CA' 'CF' 'MX' 'MD' 'MC' 'MN' 'ME' 'MA' 'MZ'
 'NA' 'NP' 'NL' 'MG' 'MW' 'MV' 'ML' 'MT' 'MH' 'MR' 'MU' 'PE' 'PH' 'PL'
 'PT' 'QA' 'RO' 'NI' 'NE' 'NG' 'MK' 'NO' 'OM' 'PK' 'PA' 'PG' 'PY' 'LU'
 'LI' 'ID' 'IR' 'IQ' 'IE' 'IL' 'IT' 'JM' 'JP' 'JO' 'KZ' 'KE' 'KR' 'XK'
 'KW' 'KG' 'LA' 'LV' 'LB' 'LS' 'LR' 'LY' 'LT' 'DZ' 'FM']
193


In [41]:
# уникальные страны для joined_dataset_pollution_2
dugr_cntrs_unique_poll = df['Country'].unique()
print(dugr_cntrs_unique_poll)
print(len(dugr_cntrs_unique_poll))

['FR' 'PL' 'RS' 'KR' 'DE' 'TR' 'SK' 'CN' 'UA' 'ES' 'BE' 'CA' 'AU' 'IN'
 'MY' 'JP' 'TH' 'BR' 'US' 'GB' 'FI' 'IS' 'IR' 'HU' 'RO' 'ZA' 'MX' 'IL'
 'LK' 'NZ' 'CL' 'ET' 'AF' 'BG' 'NL' 'IQ' 'CY' 'LA' 'DZ' 'JO' 'RU' 'RE'
 'SA' 'CH' 'LT' 'SV' 'VN' 'XK' 'CR' 'ID' 'IT' 'KW' 'CI' 'TM' 'PR' 'BH'
 'KZ' 'PK' 'MK' 'GE' 'BA' 'HR' 'TJ' 'PE' 'SE' 'NP' 'CO' 'SG' 'AR' 'DK'
 'CW' 'TW' 'EC' 'GR' 'AT' 'EE' 'BD' 'MN' 'KG' 'PH' 'PT' 'HK' 'AE' 'IE'
 'GN' 'UZ' 'NO' 'MO' 'ML' 'MM' 'GH' 'GT' 'UG' 'CZ']
94


In [42]:
# Пересечение значений для обоих датасетов. Страны есть и в joined_dataset_pollution_2 и joined_dataset_COVID_2 
dugr_same_poll_COVID = list(set(dugr_cntrs_unique_COVID) & set(dugr_cntrs_unique_poll))
print(dugr_same_poll_COVID)
print(len(dugr_same_poll_COVID))

['DE', 'ET', 'IE', 'HU', 'AF', 'CN', 'VN', 'UA', 'TJ', 'MY', 'PL', 'SK', 'NP', 'KG', 'LT', 'PT', 'IR', 'DZ', 'XK', 'ES', 'KZ', 'SG', 'FR', 'UZ', 'IS', 'HR', 'GT', 'GE', 'AE', 'MM', 'RS', 'JP', 'ZA', 'KR', 'GB', 'GR', 'EC', 'DK', 'TW', 'SV', 'IQ', 'LK', 'SE', 'SA', 'GH', 'MN', 'UG', 'CI', 'IL', 'BR', 'CR', 'CO', 'GN', 'FI', 'JO', 'PK', 'AR', 'US', 'MK', 'BA', 'IT', 'BE', 'NO', 'EE', 'BH', 'CL', 'PH', 'CH', 'PE', 'RU', 'NL', 'BG', 'NZ', 'LA', 'CZ', 'CY', 'MX', 'AT', 'TH', 'AU', 'TR', 'RO', 'BD', 'ML', 'IN', 'KW', 'CA', 'ID']
88


Делаем датасет "Работа с городами" с колонками Дуграма страны - Полное название страны- Населеные пункты COVID_PS - Населеные пункты COVID_ADMIN - Населеные пункты Pollution

In [43]:
#Разбираемся с колонкой Admin2 - результат - Admin2 есть только по стране US, США
spam = df_2[~df_2.Admin2.isnull()]
spam['Country'].unique()

array(['US'], dtype=object)

- результат - значения в колонке Admin2 есть только по стране US, США. в общий датафрейм "Работа с городами" колонку Admin2 пока не включаем. 

In [44]:
poll_towns = df['City'].unique()
type(poll_towns)
list_towns = []
for town in poll_towns:
    list_towns.append(town)
type(list_towns)
print(len(list_towns))

610


In [45]:
# образец, как работает функция по посику схожих городов
look_like = set()
all_pairs_dict= {}
poll_towns = ['Alor Setar', 'George Town', 'Ipoh', 'Johor Bahru', 'Klang', 'Kota Bharu', 'Kuala Lumpur', 'Kuantan', 'Kuching', 'Malacca', 'Miri', 'Seremban', 'Taiping']
towns_COVID_PS = ['Johor', 'Kedah', 'Kelantan', 'Melaka', 'Negeri Sembilan', 'Pahang', 'Perak', 'Perlis', 'Pulau Pinang', 'Sabah', 'Sarawak', 'Selangor', 'Terengganu', 'Unknown', 'W.P. Kuala Lumpur', 'W.P. Labuan', 'W.P. Putrajaya']
for town_pol in poll_towns:
    # сначала делаем сет из сегментов towns_COVID_PS городов
    lst_to_compare_COVID = set()
    for town_COVID in towns_COVID_PS:
        sects_Covid = town_COVID.split()
        for one_sec_COVID in sects_Covid:
            lst_to_compare_COVID.add(one_sec_COVID)
        
        sects_Covid_2 = town_COVID.split('.')
        for one_sec_COVID_2 in sects_Covid_2:
            one_sec_COVID_2 = one_sec_COVID_2.replace(' ', '')
            lst_to_compare_COVID.add(one_sec_COVID_2)
    
    # теперрь сравниваем каждый сегмент из poll_towns c каждым сегментом towns_COVID_PS
    t_split = town_pol.split()
    for one_sec_pol in t_split:
        if one_sec_pol in lst_to_compare_COVID:
            look_like.add(town_pol)
            for name in towns_COVID_PS:
                if one_sec_pol in name:
                    all_pairs_dict[town_pol] = name
            
all_pairs_dict    
    


{'Johor Bahru': 'Johor', 'Kuala Lumpur': 'W.P. Kuala Lumpur'}

In [46]:
%%time
df_towns = pd.DataFrame(columns=['Country', 'Towns_COVID_PS', 'Towns_Pollution', "Look_like"])

def exclude_town(list_towns,towns_same_poll_COVID = towns_same_poll_COVID ):
            spam_2 = []
            for town in list_towns:
                if town in towns_same_poll_COVID:
                    pass
                else:
                    spam_2.append(town)
            list_towns = spam_2 
            return list_towns
        
all_pairs_dict= {}    

def find_look_like (poll_towns, towns_COVID_PS, all_pairs_dict = all_pairs_dict):
    look_like = set()
    
    # сначала делаем сет из сегментов towns_COVID_PS городов
    lst_to_compare_COVID = set()
    for town_COVID in towns_COVID_PS:
        sects_Covid = town_COVID.split()
        for one_sec_COVID in sects_Covid:
            lst_to_compare_COVID.add(one_sec_COVID)

        sects_Covid_2 = town_COVID.split('.')
        for one_sec_COVID_2 in sects_Covid_2:
            one_sec_COVID_2 = one_sec_COVID_2.replace(' ', '')
            lst_to_compare_COVID.add(one_sec_COVID_2)
            
    # теперрь сравниваем каждый сегмент из poll_towns c каждым сегментом towns_COVID_PS        
    for town_pol in poll_towns:
        num = 1
#         print(town_pol)        
        t_split = town_pol.split()
        for one_sec_pol in t_split:
            if one_sec_pol in lst_to_compare_COVID:
                
                look_like.add(town_pol)
                for name in towns_COVID_PS:
                    if one_sec_pol in name:
                        if name in all_pairs_dict:
                            name = name + '_' + str( num)
                            num += 1
                        all_pairs_dict[name] = town_pol
#         num += 1
                        
    if len(look_like) > 0:
        look_like = list(look_like)
    else:
        look_like = '0'
    return look_like

#Если Хотим данные по проблемым городам сохранить в файл Excel - раскоментиовать код ниже и две ячейки ниже:

# num = 1
# for i in dugr_same_poll_COVID:
#     dict ={}
#     spam_df_2 = df_2.loc[(df_2['Country'] == i)]
#     spam_df_2 = spam_df_2.fillna('_')
# #     print (i, len(spam_df_2))
#     ps = spam_df_2['Province_State'].unique()
#     if len(ps) > 1:
#         spam_df = df.loc[(df['Country'] == i)]
#         spam_df = spam_df.fillna('_')
        
#         poll_towns = spam_df['City'].unique()
#         poll_towns = poll_towns.tolist()
#         poll_towns.sort()
        
#         ps = ps.tolist()
#         ps.sort()
        
#         # убираем города, которые уже итак пересекаются и находятся в двух датасетах (написание идентичное)
#         poll_towns = exclude_town(poll_towns)    
#         ps = exclude_town(ps)
        
        
#         # формируем списки похожих городов:
#         look_like = find_look_like(poll_towns, ps)

        
#         #вставляем строку данных в датафрейм df_towns 
#         df_towns.at[num, 'Country'] = i
#         df_towns.at[num, 'Towns_COVID_PS'] = ps
#         df_towns.at[num, 'Towns_Pollution'] = poll_towns
#         df_towns.at[num, 'Look_like'] = look_like
#         num +=1
        
        
# #         print(poll_towns)
# # print(df_towns)
# df_towns = pd.merge(df_towns,df_00_for_merge,on=['Country'], how='left')    
# df_towns = df_towns.reindex(columns=['Country', 'Country_Region', 'Towns_Pollution', 'Towns_COVID_PS', 'Look_like'])
# df_towns.rename(columns={'Country_Region': 'Full_name'}, inplace=True)
# df_towns        
      

Wall time: 3 ms


In [47]:

# df_towns.to_excel( 'df_towns.xlsx',
#                   sheet_name= '1'
#                 )

In [48]:
# all_pairs_dict

In [49]:
# оставялем следующий перечень, по причине не равенства области и областного центра
towns_to_rename = {
 'Mexico': 'Mexico City',
 'San Luis Potosi': 'San Luis Potosí', 
 
 'Johor': 'Johor Bahru',
 'W.P. Kuala Lumpur': 'Kuala Lumpur',
 'Sao Paulo': 'São Paulo',
 }
#'Washington': 'Washington D.C.', - зесь вопрос есть штат, есть округ

In [50]:
# df_2.loc[df_2['Province_State'] == 'Mexico']
df.loc[df['City'] == 'São Paulo']

,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region
21,2021-01-01,BR,São Paulo,pm25,303,5.0,152.0,46.0,6063.44,1609448400,Brazil
30,2021-01-01,BR,São Paulo,no2,194,0.5,21.5,4.6,148.63,1609448400,Brazil
110,2021-01-01,BR,São Paulo,dew,246,17.0,20.0,18.0,6.20,1609448400,Brazil
149,2021-01-01,BR,São Paulo,co,196,1.0,10.0,2.8,37.02,1609448400,Brazil
167,2021-01-01,BR,São Paulo,pressure,648,1015.0,1020.9,1018.0,21.62,1609448400,Brazil
...,...,...,...,...,...,...,...,...,...,...,...
2136712,2021-12-31,BR,São Paulo,no2,243,0.5,28.8,10.6,486.91,1640898000,Brazil
2136780,2021-12-31,BR,São Paulo,so2,42,0.6,2.6,0.6,1.52,1640898000,Brazil
2136820,2021-12-31,BR,São Paulo,pm25,307,5.0,168.0,21.0,6518.84,1640898000,Brazil
2136839,2021-12-31,BR,São Paulo,wind-gust,208,0.2,4.4,2.0,15.44,1640898000,Brazil


In [51]:
def multiple_replace_2(name, replace_values = towns_to_rename):
    # получаем заменяемое: подставляемое из словаря в цикле
    for i, j in replace_values.items():
        # меняем все target_str на подставляемое
        name = name.replace(i, j)
    return name

In [52]:
# меняем в датасете Квоид в Province_State. Теперь все города, ранее имевшие разное написание имеют одно написание в дух датасетах
df_2[['Province_State']] = df_2[['Province_State']].apply(multiple_replace_2)

Пробуем разобраться с городами из США, только они есть в колонке  Admin2

In [53]:
df_us_COVID = df_2[~df_2.Admin2.isnull()]
df_us_COVID

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800,US,0
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800,US,0
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800,US,0
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800,US,0
5,37165.0,Scotland,North Carolina,The United States of America,2021-01-02,34.838630,-79.475798,2522,54,0.0,2468.0,"Scotland, North Carolina, US",7242.339833,2.141158,1609534800,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000,US,0
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000,US,0
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000,US,0
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000,US,0


In [54]:
us_unique_towns_COVID_Admin2 = df_us_COVID['Admin2'].unique()
print(us_unique_towns_COVID_Admin2)
print(len(us_unique_towns_COVID_Admin2))
us_unique_towns_COVID_PS = df_us_COVID['Province_State'].unique()
print(us_unique_towns_COVID_PS)
print(len(us_unique_towns_COVID_PS))

['Rockingham' 'Rowan' 'Rutherford' ... 'Copper River' 'Chugach'
 'Out of CA']
1925
['North Carolina' 'North Dakota' 'Ohio' 'New York' 'New Mexico' 'Oregon'
 'Oklahoma' 'Pennsylvania' 'Mississippi' 'Missouri' 'Minnesota' 'Nebraska'
 'Nevada' 'New Jersey' 'New Hampshire' 'Montana' 'Virginia' 'Washington'
 'Texas' 'Utah' 'Vermont' 'Wyoming' 'Wisconsin' 'West Virginia'
 'South Dakota' 'South Carolina' 'Tennessee' 'Puerto Rico' 'Rhode Island'
 'Alabama' 'Alaska' 'Arizona' 'Colorado' 'California' 'Connecticut'
 'Delaware' 'District of Columbia' 'Florida' 'Arkansas' 'Kansas'
 'Kentucky' 'Iowa' 'Michigan' 'Massachusetts' 'Maryland' 'Louisiana'
 'Maine' 'Georgia' 'Idaho' 'Hawaii' 'Indiana' 'Illinois']
52


In [55]:
df_us_poll = df.loc[df['Country'] == 'US']
df_us_poll


,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region
23,2021-01-01,US,Salem,pm25,72,6.0,22.0,9.0,114.45,1609448400,The United States of America
28,2021-01-01,US,Salem,temperature,71,7.2,12.7,8.9,14.57,1609448400,The United States of America
36,2021-01-01,US,Salem,humidity,72,74.9,99.0,94.8,309.75,1609448400,The United States of America
43,2021-01-01,US,Jacksonville,pm25,144,17.0,71.0,38.0,844.82,1609448400,The United States of America
53,2021-01-01,US,Chicago,pressure,240,985.6,1031.5,1014.9,1010.60,1609448400,The United States of America
...,...,...,...,...,...,...,...,...,...,...,...
2136833,2021-12-31,US,Tallahassee,humidity,72,51.0,100.0,88.0,1298.34,1640898000,The United States of America
2136841,2021-12-31,US,Seattle,pressure,264,1007.8,1024.7,1016.9,205.26,1640898000,The United States of America
2136843,2021-12-31,US,Jackson,humidity,72,54.0,95.3,85.5,1343.64,1640898000,The United States of America
2136844,2021-12-31,US,Seattle,no2,68,1.9,28.1,10.7,580.07,1640898000,The United States of America


In [56]:
us_unique_towns_poll = df_us_poll['City'].unique()
print(us_unique_towns_poll)
print(len(us_unique_towns_poll))

['Salem' 'Jacksonville' 'Chicago' 'Providence' 'Los Angeles' 'Fresno'
 'San Jose' 'Miami' 'Madison' 'San Francisco' 'Las Vegas'
 'Washington D.C.' 'Albuquerque' 'Staten Island' 'Houston' 'Tucson'
 'Raleigh' 'Oakland' 'Manhattan' 'Honolulu' 'Queens' 'Seattle'
 'Oklahoma City' 'Nashville' 'Philadelphia' 'Saint Paul' 'The Bronx'
 'Milwaukee' 'Brooklyn' 'Denver' 'Hartford' 'San Diego' 'Springfield'
 'Columbus' 'Salt Lake City' 'San Antonio' 'Little Rock' 'Sacramento'
 'Phoenix' 'Indianapolis' 'El Paso' 'Detroit' 'Portland' 'Boston'
 'Baltimore' 'Charlotte' 'Jackson' 'Atlanta' 'Austin' 'Boise' 'Fort Worth'
 'Dallas' 'Omaha' 'Richmond' 'Memphis' 'Tallahassee']
56


In [57]:
#Города из датасет  pollution есть в датасет COVID
towns_us_not_included_1 = list(set(us_unique_towns_poll) - set(us_unique_towns_COVID_Admin2))
towns_us_not_included_1

['Memphis',
 'The Bronx',
 'Saint Paul',
 'Jacksonville',
 'Washington D.C.',
 'Salt Lake City',
 'Manhattan',
 'Atlanta',
 'Springfield',
 'Seattle',
 'Nashville',
 'Indianapolis',
 'Brooklyn',
 'Staten Island',
 'Portland',
 'San Jose',
 'Albuquerque',
 'Oklahoma City',
 'Little Rock',
 'Chicago',
 'Fort Worth',
 'Detroit',
 'San Antonio',
 'Tallahassee',
 'Phoenix',
 'Las Vegas',
 'Boston',
 'Tucson',
 'Omaha']

In [58]:
#Города которые есть и в датасет  pollution есть в датасет COVID колонка Province_State
towns_same_poll_COVID_PS_US = list(set(us_unique_towns_poll) & set(us_unique_towns_COVID_PS))
print(towns_same_poll_COVID_PS_US)
print(len(towns_same_poll_COVID_PS_US))

[]
0


In [59]:
#Города которые есть и в датасет  pollution есть в датасет COVID колонка Admin2
towns_same_poll_COVID_Admin_US = list(set(us_unique_towns_poll) & set(us_unique_towns_COVID_Admin2))
print(towns_same_poll_COVID_Admin_US)
print(len(towns_same_poll_COVID_Admin_US))

['San Diego', 'Philadelphia', 'El Paso', 'Providence', 'Raleigh', 'Charlotte', 'Oakland', 'Dallas', 'Denver', 'Sacramento', 'Honolulu', 'Baltimore', 'Austin', 'San Francisco', 'Columbus', 'Hartford', 'Jackson', 'Milwaukee', 'Madison', 'Boise', 'Salem', 'Los Angeles', 'Fresno', 'Richmond', 'Houston', 'Queens', 'Miami']
27


In [60]:
#Города которые есть в датасет  pollution но их НЕТ в датасет COVID колонка Admin2 и в колонке Province_State
towns_NOT_same_poll_COVID_Admin_US = list(set(us_unique_towns_poll) - set(towns_same_poll_COVID_Admin_US) - set(towns_same_poll_COVID_PS_US))
print(towns_NOT_same_poll_COVID_Admin_US)
print(len(towns_NOT_same_poll_COVID_Admin_US))

['Staten Island', 'Memphis', 'Brooklyn', 'Portland', 'San Jose', 'The Bronx', 'Oklahoma City', 'Little Rock', 'Saint Paul', 'Jacksonville', 'Washington D.C.', 'Salt Lake City', 'Chicago', 'Manhattan', 'Fort Worth', 'Detroit', 'Atlanta', 'Springfield', 'Las Vegas', 'Seattle', 'San Antonio', 'Nashville', 'Tallahassee', 'Boston', 'Phoenix', 'Indianapolis', 'Albuquerque', 'Tucson', 'Omaha']
29


In [61]:
all_pairs_dict.clear()
like_same_towns_US = find_look_like(us_unique_towns_COVID_Admin2, towns_NOT_same_poll_COVID_Admin_US)

In [62]:
like_same_towns_US = find_look_like(us_unique_towns_COVID_PS, towns_NOT_same_poll_COVID_Admin_US)

In [63]:
all_pairs_dict

{'Washington D.C.': 'Washington',
 'San Jose': 'San Juan',
 'San Antonio': 'San Juan',
 'San Jose_1': 'San Benito',
 'San Antonio_2': 'San Benito',
 'The Bronx': 'Bronx',
 'Salt Lake City': 'Lake',
 'Oklahoma City': 'Oklahoma',
 'Oklahoma City_1': 'Oklahoma',
 'Salt Lake City_2': 'Baltimore City',
 'Salt Lake City_1': 'Bear Lake',
 'Little Rock': 'Rock',
 'Fort Worth': 'Worth',
 'Staten Island': 'Island',
 'Las Vegas': 'Las Marias',
 'Las Vegas_1': 'Las Animas',
 'Fort Worth_1': 'Fort Bend',
 'Staten Island_1': 'Rhode Island',
 'Little Rock_1': 'Rock Island',
 'Staten Island_2': 'Rock Island',
 'Washington D.C._1': 'Washington'}

Вывод менять названия городов в колонках Admin и в Province_State в датасете COVID через фукнцию  multiply replace #не имеет смысла 

Остается вопрос - какое значение из Admin2 и в Province_State в датасете COVID оставить для сравнения с датасет Pollution.
ВОзможный ответ: оставлять то значение, которое есть в уникальном списке городов Pollution

в США совпадения с us_unique_towns_poll есть только в колонке Admin2

Вывод: в датафрейм COVID ставим в колонку Admin2 значения равные колонке  Province_State 

In [64]:
df_2.loc[(df_2['Country'] != 'US' ) , 'Admin2'] =  df_2['Province_State']  
df_2    

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
0,NaN,NaN,NaN,Afghanistan,2021-01-02,33.939110,67.709953,52513,2201,41727.0,8585.0,Afghanistan,134.896578,4.191343,1609534800,AF,0
1,37157.0,Rockingham,North Carolina,The United States of America,2021-01-02,36.395931,-79.775048,4432,25,0.0,4407.0,"Rockingham, North Carolina, US",4869.794528,0.564079,1609534800,US,0
2,37159.0,Rowan,North Carolina,The United States of America,2021-01-02,35.640039,-80.524466,9017,168,0.0,8849.0,"Rowan, North Carolina, US",6346.067226,1.863147,1609534800,US,0
3,37161.0,Rutherford,North Carolina,The United States of America,2021-01-02,35.401142,-81.921121,4337,95,0.0,4242.0,"Rutherford, North Carolina, US",6470.333736,2.190454,1609534800,US,0
4,37163.0,Sampson,North Carolina,The United States of America,2021-01-02,34.988204,-78.369141,4912,61,0.0,4851.0,"Sampson, North Carolina, US",7731.658560,1.241857,1609534800,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454381,17123.0,Marshall,Illinois,The United States of America,2021-12-31,41.032790,-89.345104,1927,24,NaN,NaN,"Marshall, Illinois, US",16847.350935,1.245459,1640898000,US,0
1454382,17125.0,Mason,Illinois,The United States of America,2021-12-31,40.239173,-89.918245,2704,53,NaN,NaN,"Mason, Illinois, US",20241.036006,1.960059,1640898000,US,0
1454383,17127.0,Massac,Illinois,The United States of America,2021-12-31,37.220784,-88.710366,2631,58,NaN,NaN,"Massac, Illinois, US",19103.979088,2.204485,1640898000,US,0
1454384,17095.0,Knox,Illinois,The United States of America,2021-12-31,40.931488,-90.213303,8838,184,NaN,NaN,"Knox, Illinois, US",17783.053985,2.081919,1640898000,US,0


In [65]:
df_2.loc[(df_2['Country'] == 'RU' )]

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
2118,NaN,Moscow,Moscow,Russia,2021-01-02,55.750446,37.617494,815676,11280,633702.0,170694.0,"Moscow, Russia",6522.033239,1.382902,1609534800,RU,1
2119,NaN,Moscow Oblast,Moscow Oblast,Russia,2021-01-02,55.504316,38.035393,156898,2914,113744.0,40240.0,"Moscow Oblast, Russia",2091.029582,1.857258,1609534800,RU,0
2120,NaN,Murmansk Oblast,Murmansk Oblast,Russia,2021-01-02,68.000042,33.999915,36363,655,33259.0,2449.0,"Murmansk Oblast, Russia",4825.514195,1.801282,1609534800,RU,0
2121,NaN,Nenets Autonomous Okrug,Nenets Autonomous Okrug,Russia,2021-01-02,68.275572,57.168638,796,0,697.0,99.0,"Nenets Autonomous Okrug, Russia",1809.214265,0.000000,1609534800,RU,0
2122,NaN,Nizhny Novgorod Oblast,Nizhny Novgorod Oblast,Russia,2021-01-02,55.471803,44.091159,70321,1663,62602.0,6056.0,"Nizhny Novgorod Oblast, Russia",2173.922452,2.364870,1609534800,RU,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452629,NaN,Vladimir Oblast,Vladimir Oblast,Russia,2021-12-31,56.050334,40.656163,81040,2922,NaN,NaN,"Vladimir Oblast, Russia",5879.549051,3.605627,1640898000,RU,0
1452630,NaN,Volgograd Oblast,Volgograd Oblast,Russia,2021-12-31,49.604834,44.290358,128929,5510,NaN,NaN,"Volgograd Oblast, Russia",5113.640871,4.273670,1640898000,RU,0
1452631,NaN,Vologda Oblast,Vologda Oblast,Russia,2021-12-31,60.039146,43.121521,99811,2951,NaN,NaN,"Vologda Oblast, Russia",8482.360250,2.956588,1640898000,RU,0
1452632,NaN,Voronezh Oblast,Voronezh Oblast,Russia,2021-12-31,50.980039,40.150651,193855,6878,NaN,NaN,"Voronezh Oblast, Russia",8306.524042,3.548013,1640898000,RU,0


# Сравнение датасетов COVID и Pollution будет идти по совпадению города в колонке Admin2

Убираем строки с пустым знаяениеме в колонке Admin2 в датасете COVID - их не с чем сравнивать 

In [66]:
df_2['Admin2'].isna().sum()

63766

In [67]:
df_2 = df_2.dropna(subset = ['Admin2'])
df_2['Admin2'].isna().sum()

0

Теперь задача оставить в датасете только те строки, которые в колонке Admin2 имеют названия городав, встречающиеся в датасете Polluton в колонке City ( мы уже ранее получили список уникальных имен городов в датасете Pollution - poll_uniq_towns)

In [68]:
df_2_1 = df_2.loc[df_2['Admin2'].isin(poll_uniq_towns)]
df_2_1

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll
30,37153.0,Richmond,North Carolina,The United States of America,2021-01-02,35.004177,-79.744910,2630,47,0.0,2583.0,"Richmond, North Carolina, US",5866.738049,1.787072,1609534800,US,1
34,37099.0,Jackson,North Carolina,The United States of America,2021-01-02,35.288143,-83.140934,2259,18,0.0,2241.0,"Jackson, North Carolina, US",5141.335518,0.796813,1609534800,US,1
41,37115.0,Madison,North Carolina,The United States of America,2021-01-02,35.854027,-82.713464,797,19,0.0,778.0,"Madison, North Carolina, US",3663.525626,2.383940,1609534800,US,1
83,39009.0,Athens,Ohio,The United States of America,2021-01-02,39.334256,-82.042786,3101,25,0.0,3076.0,"Athens, Ohio, US",4746.888729,0.806192,1609534800,US,1
128,36041.0,Hamilton,New York,The United States of America,2021-01-02,43.661466,-74.497220,112,1,0.0,111.0,"Hamilton, New York, US",2536.231884,0.892857,1609534800,US,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454196,18103.0,Miami,Indiana,The United States of America,2021-12-31,40.769410,-86.045250,7188,123,NaN,NaN,"Miami, Indiana, US",20238.765627,1.711185,1640898000,US,1
1454310,16065.0,Madison,Idaho,The United States of America,2021-12-31,43.781410,-111.653847,9362,42,NaN,NaN,"Madison, Idaho, US",23459.543438,0.448622,1640898000,US,1
1454324,17065.0,Hamilton,Illinois,The United States of America,2021-12-31,38.081636,-88.538834,1540,25,NaN,NaN,"Hamilton, Illinois, US",18974.864465,1.623377,1640898000,US,1
1454361,17077.0,Jackson,Illinois,The United States of America,2021-12-31,37.785952,-89.382326,9483,101,NaN,NaN,"Jackson, Illinois, US",16710.132159,1.065064,1640898000,US,1


In [69]:
print(len(poll_uniq_towns))
rest_unique_towns_COVID = df_2_1['Admin2'].unique()
print(len(rest_unique_towns_COVID))

610
91


То есть в отфильтрованном датаесете COVID мы имеем только 64000 строк по 91 населенному пукту

Теперь фильруем датасет Pollution по тем 91 городам, которые остались в датасете COVID

In [70]:
df

,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region
0,2021-01-01,FR,Saint-Étienne,temperature,96,1.0,2.0,1.0,1.68,1609448400,France
1,2021-01-01,PL,Katowice,so2,94,3.1,17.6,8.8,106.62,1609448400,Poland
2,2021-01-01,RS,Novi Sad,so2,71,3.3,30.5,9.0,224.44,1609448400,Serbia
3,2021-01-01,KR,Busan,humidity,637,39.0,68.0,51.0,588.90,1609448400,South Korea
4,2021-01-01,DE,Freiburg,wind-speed,43,0.2,3.7,1.9,11.93,1609448400,Germany
...,...,...,...,...,...,...,...,...,...,...,...
2136876,2021-12-31,BA,Tuzla,o3,117,0.4,17.8,3.4,147.35,1640898000,Bosnia and Herzegovina
2136877,2021-12-31,AT,Linz,dew,39,6.0,10.4,9.5,12.66,1640898000,Austria
2136878,2021-12-31,AU,Darwin,no2,69,0.3,3.6,1.1,11.98,1640898000,Australia
2136879,2021-12-31,ES,Oviedo,humidity,261,31.0,91.3,61.0,1623.29,1640898000,Spain


In [71]:
df_0_2 = df.loc[df['City'].isin(rest_unique_towns_COVID)]
df_0_2

,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region
11,2021-01-01,ES,Murcia,humidity,37,33.5,86.5,49.8,2586.99,1609448400,Spain
19,2021-01-01,JP,Kochi,so2,3,1.5,1.5,1.5,0.00,1609448400,Japan
21,2021-01-01,BR,São Paulo,pm25,303,5.0,152.0,46.0,6063.44,1609448400,Brazil
23,2021-01-01,US,Salem,pm25,72,6.0,22.0,9.0,114.45,1609448400,The United States of America
26,2021-01-01,CN,Tianjin,pressure,360,1024.0,1029.0,1026.0,15.42,1609448400,China
...,...,...,...,...,...,...,...,...,...,...,...
2136843,2021-12-31,US,Jackson,humidity,72,54.0,95.3,85.5,1343.64,1640898000,The United States of America
2136856,2021-12-31,JP,Toyama,temperature,162,-2.5,1.3,-0.8,8.36,1640898000,Japan
2136857,2021-12-31,BR,São Paulo,dew,215,17.0,20.0,18.0,3.77,1640898000,Brazil
2136868,2021-12-31,US,Jackson,pressure,72,1009.3,1014.7,1012.1,21.32,1640898000,The United States of America


In [72]:
#делаем проверку количесва оставшихся в датасете Pollution городов:
rest_unique_towns_Poll = df_0_2['City'].unique()
print(len(rest_unique_towns_Poll))
# если 91 - то все ОК

91


In [73]:
df = df_0_2
df_2 = df_2_1

Делаем колонку Admin2 + Absolute_time. Чтобы получить уникальный ключ в двух датасетах, по которому потом будем соеденять два датасета

In [74]:
# df_2_1 = df_2
df_2['Main_key']= df_2["Admin2"].astype(str) +"_"+ df_2["Absolute_time"].astype(str)

# df_2_1['Main_key'] = df_2_1['Admin2'] + str(df_2_1['Absolute_time']) 
# df_2['Main_key'] = df_2.apply(lambda row: row.str('Admin2') + '_' + row.str('Absolute_time'), axis = 1)
df_2

,FIPS,Admin2,Province_State,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Absolute_time,Country,intersection_poll,Main_key
30,37153.0,Richmond,North Carolina,The United States of America,2021-01-02,35.004177,-79.744910,2630,47,0.0,2583.0,"Richmond, North Carolina, US",5866.738049,1.787072,1609534800,US,1,Richmond_1609534800
34,37099.0,Jackson,North Carolina,The United States of America,2021-01-02,35.288143,-83.140934,2259,18,0.0,2241.0,"Jackson, North Carolina, US",5141.335518,0.796813,1609534800,US,1,Jackson_1609534800
41,37115.0,Madison,North Carolina,The United States of America,2021-01-02,35.854027,-82.713464,797,19,0.0,778.0,"Madison, North Carolina, US",3663.525626,2.383940,1609534800,US,1,Madison_1609534800
83,39009.0,Athens,Ohio,The United States of America,2021-01-02,39.334256,-82.042786,3101,25,0.0,3076.0,"Athens, Ohio, US",4746.888729,0.806192,1609534800,US,1,Athens_1609534800
128,36041.0,Hamilton,New York,The United States of America,2021-01-02,43.661466,-74.497220,112,1,0.0,111.0,"Hamilton, New York, US",2536.231884,0.892857,1609534800,US,1,Hamilton_1609534800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454196,18103.0,Miami,Indiana,The United States of America,2021-12-31,40.769410,-86.045250,7188,123,NaN,NaN,"Miami, Indiana, US",20238.765627,1.711185,1640898000,US,1,Miami_1640898000
1454310,16065.0,Madison,Idaho,The United States of America,2021-12-31,43.781410,-111.653847,9362,42,NaN,NaN,"Madison, Idaho, US",23459.543438,0.448622,1640898000,US,1,Madison_1640898000
1454324,17065.0,Hamilton,Illinois,The United States of America,2021-12-31,38.081636,-88.538834,1540,25,NaN,NaN,"Hamilton, Illinois, US",18974.864465,1.623377,1640898000,US,1,Hamilton_1640898000
1454361,17077.0,Jackson,Illinois,The United States of America,2021-12-31,37.785952,-89.382326,9483,101,NaN,NaN,"Jackson, Illinois, US",16710.132159,1.065064,1640898000,US,1,Jackson_1640898000


In [75]:
df['Main_key']= df["City"].astype(str) +"_"+ df["Absolute_time"].astype(str)
df

C:\Users\aanal\AppData\Local\Temp/ipykernel_6536/3109886204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Main_key']= df["City"].astype(str) +"_"+ df["Absolute_time"].astype(str)


,Date,Country,City,Specie,count,min,max,median,variance,Absolute_time,Country_Region,Main_key
11,2021-01-01,ES,Murcia,humidity,37,33.5,86.5,49.8,2586.99,1609448400,Spain,Murcia_1609448400
19,2021-01-01,JP,Kochi,so2,3,1.5,1.5,1.5,0.00,1609448400,Japan,Kochi_1609448400
21,2021-01-01,BR,São Paulo,pm25,303,5.0,152.0,46.0,6063.44,1609448400,Brazil,São Paulo_1609448400
23,2021-01-01,US,Salem,pm25,72,6.0,22.0,9.0,114.45,1609448400,The United States of America,Salem_1609448400
26,2021-01-01,CN,Tianjin,pressure,360,1024.0,1029.0,1026.0,15.42,1609448400,China,Tianjin_1609448400
...,...,...,...,...,...,...,...,...,...,...,...,...
2136843,2021-12-31,US,Jackson,humidity,72,54.0,95.3,85.5,1343.64,1640898000,The United States of America,Jackson_1640898000
2136856,2021-12-31,JP,Toyama,temperature,162,-2.5,1.3,-0.8,8.36,1640898000,Japan,Toyama_1640898000
2136857,2021-12-31,BR,São Paulo,dew,215,17.0,20.0,18.0,3.77,1640898000,Brazil,São Paulo_1640898000
2136868,2021-12-31,US,Jackson,pressure,72,1009.3,1014.7,1012.1,21.32,1640898000,The United States of America,Jackson_1640898000


Тепреь мы имеем в датасетах только те данные с которыми будем работать в дальнейшем

Сохраняем датасеты в промежуточные файлы

In [76]:
#сохраняем датасет под индексом 3
df.to_csv('joined_dataset_pollution_3.csv', index=False)
df_2.to_csv('joined_dataset_COVID_3.csv', index=False)

Видим проблему с названиями городов - разные населенные пункты имеют одинаковое названия, отличаются по расположению в разных районах/штатах или в разных странах

In [77]:
# # функция выбора значения на оснве двух столбцов
# def choose_from_two_columns(r):
#     if r.Admin2 in    
    

In [78]:
# In [104]: def total(r):
#      ...:     if r.a == r.b:
#      ...:         return r.a
#      ...:     elif (r.a == 'fruit') and (r.b == 'both'):
#      ...:         return r.a
#      ...:     return 'Unknown'
#      ...:

# In [105]: df = pd.DataFrame({'a': ['fruit', 'vegetable', 'vegetable', 'fruit'], 'b': ['fruit', 'vegetable', 'both', 'both']})

# In [106]: df
# Out[106]:
#            a          b
# 0      fruit      fruit
# 1  vegetable  vegetable
# 2  vegetable       both
# 3      fruit       both

# In [107]: df['total'] = df.apply(total, axis=1)

# In [108]: df